# Coordinate Alignment

Since linopy builds on xarray, coordinate alignment matters when combining variables or expressions that live on different coordinates.

linopy uses **strict, operation-dependent defaults** that prevent silent data loss and ambiguous fill behavior:

| Operation | Default | On mismatch |
|-----------|---------|-------------|
| `+`, `-` | `"exact"` | `ValueError` — coordinates must match |
| `*`, `/` | `"inner"` | Intersection — natural filtering |
| `<=`, `>=`, `==` (DataArray RHS) | `"exact"` | `ValueError` — coordinates must match |

When you need to combine operands with mismatched coordinates, use the named methods (`.add()`, `.sub()`, `.mul()`, `.div()`, `.le()`, `.ge()`, `.eq()`) with an explicit `join=` parameter.

This convention is inspired by [pyoframe](https://github.com/Bravos-Power/pyoframe).

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import linopy

## Matching Coordinates — The Default Case

When two operands share the same coordinates on every shared dimension, all operators work directly. No special handling is needed.

In [ ]:
m = linopy.Model()

time = pd.RangeIndex(5, name="time")
x = m.add_variables(lower=0, coords=[time], name="x")
y = m.add_variables(lower=0, coords=[time], name="y")

# Same coordinates — works fine
x + y

In [ ]:
factor = xr.DataArray([2, 3, 4, 5, 6], dims=["time"], coords={"time": time})
x * factor

## Broadcasting (Different Dimensions)

Alignment only checks **shared** dimensions. If operands have different dimension names, they expand (broadcast) as in xarray — this is unaffected by the alignment convention.

This works in both directions: a constant with extra dimensions expands the expression, and an expression with extra dimensions expands over the constant.

In [ ]:
techs = pd.Index(["solar", "wind", "gas"], name="tech")
cost = xr.DataArray([1.0, 0.5, 3.0], dims=["tech"], coords={"tech": techs})

# x has dim "time", cost has dim "tech" — no shared dim, pure broadcast
x * cost  # -> (time, tech)

In [ ]:
# Constant with MORE dimensions than the expression — also broadcasts
w = m.add_variables(lower=0, coords=[techs], name="w")  # dims: (tech,)
time_profile = xr.DataArray(
    [[1, 2], [3, 4], [5, 6]],
    dims=["tech", "time"],
    coords={"tech": techs, "time": [0, 1]},
)

# w has dim "tech", time_profile has dims ("tech", "time")
# "time" is extra — it expands the expression via broadcasting
w + time_profile  # -> (tech, time)

## Addition / Subtraction: `"exact"` Default

When operands have different coordinates on a shared dimension, `+` and `-` raise a `ValueError`. This prevents silent data loss or ambiguous fill behavior.

In [ ]:
subset_time = pd.RangeIndex(3, name="time")
y_short = m.add_variables(lower=0, coords=[subset_time], name="y_short")

# x has 5 time steps, y_short has 3 — coordinates don't match
try:
    x + y_short
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Same for adding a constant DataArray with mismatched coordinates
partial_const = xr.DataArray([10, 20, 30], dims=["time"], coords={"time": [0, 1, 2]})

try:
    x + partial_const
except ValueError as e:
    print("ValueError:", e)

## Multiplication / Division: `"inner"` Default

Multiplication by a parameter array is a natural filtering operation — like applying an availability factor to a subset of time steps. The result is restricted to the **intersection** of coordinates. No fill values are needed.

In [ ]:
partial_factor = xr.DataArray([2, 3, 4], dims=["time"], coords={"time": [0, 1, 2]})

# x has time 0-4, partial_factor has time 0-2
# Inner join: result restricted to time 0, 1, 2
x * partial_factor

In [ ]:
# Disjoint coordinates: no intersection -> empty result
z = m.add_variables(lower=0, coords=[pd.RangeIndex(5, 10, name="time")], name="z")
disjoint_factor = xr.DataArray(
    [1, 2, 3, 4, 5], dims=["time"], coords={"time": range(5)}
)

z * disjoint_factor

## Named Methods with `join=`

When you intentionally want to combine operands with mismatched coordinates, use the named methods with an explicit `join=` parameter. This makes the alignment intent clear in the code.

### Setup: Overlapping but Non-Identical Coordinates

In [ ]:
m2 = linopy.Model()

i_a = pd.Index([0, 1, 2], name="i")
i_b = pd.Index([1, 2, 3], name="i")

a = m2.add_variables(coords=[i_a], name="a")
b = m2.add_variables(coords=[i_b], name="b")

`a` has coordinates i=[0, 1, 2] and `b` has i=[1, 2, 3]. They overlap at i=1 and i=2 but are not identical, so `a + b` raises a `ValueError`.

**Inner join** — only shared coordinates (i=1, 2):

In [ ]:
a.add(b, join="inner")

**Outer join** — union of coordinates (i=0, 1, 2, 3). Where one operand is missing, it drops out of the sum (fill with zero):

In [ ]:
a.add(b, join="outer")

**Left join** — keep left operand's coordinates (i=0, 1, 2):

In [ ]:
a.add(b, join="left")

**Right join** — keep right operand's coordinates (i=1, 2, 3):

In [ ]:
a.add(b, join="right")

**Override** — positional alignment, ignore coordinate labels. The result uses the left operand's coordinates:

In [ ]:
a.add(b, join="override")

### Multiplication with `join=`

The same `join=` parameter works on `.mul()` and `.div()`. Since multiplication defaults to `"inner"`, you only need explicit `join=` when you want a different mode:

In [ ]:
const = xr.DataArray([2, 3, 4], dims=["i"], coords={"i": [1, 2, 3]})

# Default inner join — intersection of i=[0,1,2] and i=[1,2,3]
a * const

## Constraints with DataArray RHS

Constraint operators (`<=`, `>=`, `==`) with a DataArray right-hand side also default to `"exact"` — coordinates on shared dimensions must match. Use `.le()`, `.ge()`, `.eq()` with `join=` to control alignment.

**Dimension rules for constraint RHS:**
- The RHS may have **fewer** dimensions than the expression — the bound broadcasts. This is the standard way to apply a per-tech capacity across all time steps.
- The RHS must **not** have **more** dimensions than the expression. An expression with `dims=(tech,)` defines one variable per tech; an RHS with `dims=(time, tech)` would create redundant constraints on the same variable, which is almost always a mistake.

Note: this is different from arithmetic, where a constant with extra dims freely expands the expression. For constraints, the expression defines the problem structure.

In [ ]:
# RHS with fewer dimensions — broadcasts (works fine)
m3 = linopy.Model()
hours = pd.RangeIndex(24, name="hour")
techs = pd.Index(["solar", "wind", "gas"], name="tech")
gen = m3.add_variables(lower=0, coords=[hours, techs], name="gen")

capacity = xr.DataArray([100, 80, 50], dims=["tech"], coords={"tech": techs})
m3.add_constraints(
    gen <= capacity, name="capacity_limit"
)  # capacity broadcasts over hour

In [ ]:
# RHS with matching coordinates — works fine
full_rhs = xr.DataArray(np.arange(5, dtype=float), dims=["time"], coords={"time": time})
con = x <= full_rhs
con

In [ ]:
# RHS with mismatched coordinates — raises ValueError
partial_rhs = xr.DataArray([10, 20, 30], dims=["time"], coords={"time": [0, 1, 2]})

try:
    x <= partial_rhs
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Use .le() with join="inner" — constraint only at the intersection
x.to_linexpr().le(partial_rhs, join="inner")

In [ ]:
# Use .le() with join="left" — constraint at all of x's coordinates,
# NaN where RHS is missing (no constraint at those positions)
x.to_linexpr().le(partial_rhs, join="left")

In [ ]:
# RHS with MORE dimensions than expression — raises ValueError
y_tech = m.add_variables(lower=0, coords=[techs], name="y_tech")  # dims: (tech,)
rhs_extra_dims = xr.DataArray(
    np.ones((5, 3)), dims=["time", "tech"], coords={"time": time, "tech": techs}
)

try:
    y_tech <= rhs_extra_dims  # "time" is not in the expression
except ValueError as e:
    print("ValueError:", e)

## Practical Example

Consider a generation dispatch model where solar availability is a partial factor and a minimum demand constraint only applies during peak hours.

In [ ]:
m4 = linopy.Model()

hours = pd.RangeIndex(24, name="hour")
techs = pd.Index(["solar", "wind", "gas"], name="tech")

gen = m4.add_variables(lower=0, coords=[hours, techs], name="gen")

Capacity limits apply to all hours and techs. The `capacity` DataArray has only the `tech` dimension — it broadcasts over `hour` (no shared dimension to conflict):

In [ ]:
capacity = xr.DataArray([100, 80, 50], dims=["tech"], coords={"tech": techs})
m4.add_constraints(gen <= capacity, name="capacity_limit")

Solar availability is a factor that covers all 24 hours. Since coordinates match exactly, multiplication with `*` works directly:

In [ ]:
solar_avail = np.zeros(24)
solar_avail[6:19] = np.sin(np.linspace(0, np.pi, 13))
solar_availability = xr.DataArray(solar_avail, dims=["hour"], coords={"hour": hours})

solar_gen = gen.sel(tech="solar")
m4.add_constraints(solar_gen <= 100 * solar_availability, name="solar_avail")

Peak demand of 120 MW must be met only during hours 8-20. The demand array covers a subset of hours. Use `.ge()` with `join="inner"` to restrict the constraint to just those hours:

In [ ]:
peak_hours = pd.RangeIndex(8, 21, name="hour")
peak_demand = xr.DataArray(
    np.full(len(peak_hours), 120.0), dims=["hour"], coords={"hour": peak_hours}
)

total_gen = gen.sum("tech")

# Constraint only at peak hours (intersection)
m4.add_constraints(total_gen.ge(peak_demand, join="inner"), name="peak_demand")

Selecting the correct subset of the variable produces the same result, and is arguably more readable:

In [ ]:
# Constraint only at peak hours (intersection)
m4.add_constraints(
    total_gen.sel(hour=peak_hours) >= peak_demand, name="peak_demand_sel"
)

## Migrating from Previous Versions

Previous versions of linopy used a **shape-dependent heuristic** for coordinate alignment. The behavior depended on whether operands happened to have the same shape, and was inconsistent between `Variable` and `LinearExpression`:

| Condition | Old behavior | New behavior |
|-----------|-------------|-------------|
| Same shape, same coordinates | Works correctly | Works correctly (no change) |
| Same shape, **different** coordinates, `+`/`-` | `"override"` — positional alignment (**bug-prone**) | `"exact"` — raises `ValueError` |
| Same shape, **different** coordinates, `*`/`/` | Buggy (crashes or produces garbage) | `"inner"` — intersection |
| Different shape, expr + expr | `"outer"` — union of coordinates | `"exact"` — raises `ValueError` |
| Different shape, expr + constant | `"left"` — keeps expression coords, fills missing with 0 | `"exact"` — raises `ValueError` |
| Different shape, expr * constant | Buggy (crashes for `LinearExpression`, produces garbage for `Variable`) | `"inner"` — intersection |
| Constraint with mismatched DataArray RHS | Same-shape: `"override"` (positional); different-shape: `"left"` (fills missing RHS with 0) | `"exact"` — raises `ValueError` |

### Why the change?

The old heuristic caused several classes of bugs:

1. **Silent positional alignment**: When two operands happened to have the same shape but entirely different coordinates (e.g., `x(time=[0,1,2]) + z(time=[5,6,7])`), they were matched by position — giving a wrong result with no warning.

2. **Non-associative addition**: `(y + factor) + x` could give a different result than `y + (x + factor)` because `"left"` for expr+constant dropped the constant's extra coordinates before they could be recovered by a subsequent addition.

3. **Broken multiplication**: Multiplying a `LinearExpression` by a DataArray with mismatched coordinates would crash with an `AssertionError`. Multiplying a `Variable` by such a DataArray produced a result with misaligned coefficients and variable references.

### How to update your code

If your code combines operands with **mismatched coordinates** and you relied on the old behavior, you'll now get a `ValueError` (for `+`/`-`) or a smaller result (for `*`/`/`). Here's how to migrate:

**Addition with mismatched coordinates** — expr+expr previously used `"outer"`, expr+constant used `"left"`. Both now raise `ValueError`:

```python
# Old code (worked silently):
result = x + y_short          # different-size expr+expr → was "outer"
result = x + partial_const    # expr + constant → was "left"

# New code — be explicit about the join:
result = x.add(y_short, join="outer")        # union of coordinates
result = x.add(partial_const, join="left")   # keep x's coordinates, fill 0
```

**Same-shape but different coordinates** — previously matched by position (`"override"`) for addition. Now raises `ValueError` for `+`/`-`, gives intersection for `*`/`/`:

```python
# Old code (silently matched positions — likely a bug!):
x_abc = m.add_variables(coords=[["a", "b", "c"]], name="x_abc")
y_def = m.add_variables(coords=[["d", "e", "f"]], name="y_def")
result = x_abc + y_def   # Old: positional match → New: ValueError

# If you really want positional matching (rare):
result = x_abc.add(y_def, join="override")
```

**Multiplication with mismatched coordinates** — previously broken (crash or garbage). Now uses `"inner"` (intersection):

```python
# Old code — would crash (LinExpr) or produce garbage (Variable):
x * partial_factor   # x has 5 coords, partial_factor has 3

# New code — result has 3 entries (intersection). This now works correctly!
# If you need to keep all of x's coordinates (zero-fill missing):
x.mul(partial_factor, join="left")
```

**Constraints with mismatched DataArray RHS** — previously used positional alignment (same shape) or `"left"` with 0-fill (different shape). Now raises `ValueError`:

```python
# Old code:
con = x <= partial_rhs   # Old: "left" (fill 0) or "override" → New: ValueError

# New code — be explicit:
con = x.to_linexpr().le(partial_rhs, join="left")    # keep x's coords, NaN fill
con = x.to_linexpr().le(partial_rhs, join="inner")   # intersection only
```

## Summary

### Default Behavior

| Context | Default `join` | Behavior |
|---------|---------------|----------|
| Arithmetic operators (`+`, `-`) | `"exact"` | Coordinates must match on shared dims; raises `ValueError` on mismatch |
| Arithmetic operators (`*`, `/`) | `"inner"` | Intersection of coordinates on shared dims; no fill needed |
| Constraint operators (`<=`, `>=`, `==`) with DataArray RHS | `"exact"` | Coordinates must match on shared dims; raises `ValueError` on mismatch |

### Extra Dimensions (Broadcasting)

| Context | Extra dims on constant/RHS | Extra dims on expression |
|---------|--------------------------|------------------------|
| Arithmetic (`+`, `-`, `*`, `/`) | Expands the expression (standard xarray broadcast) | Expands over the constant |
| Constraint RHS (`<=`, `>=`, `==`) | **Forbidden** — raises `ValueError` | RHS broadcasts over expression's extra dims |

### All Join Modes

| `join` | Coordinates | Fill behavior |
|--------|------------|---------------|
| `"exact"` (default for `+`, `-`, constraints) | Must match exactly | Raises `ValueError` if different |
| `"inner"` (default for `*`, `/`) | Intersection only | No fill needed |
| `"outer"` | Union | Fill with zero (arithmetic) or `NaN` (constraint RHS) |
| `"left"` | Left operand's | Fill right with zero (arithmetic) or `NaN` (constraint RHS) |
| `"right"` | Right operand's | Fill left with zero |
| `"override"` | Left operand's (positional) | Positional alignment, ignores coordinate labels |

### Quick Reference

| Operation | Matching coords | Mismatched coords |
|-----------|----------------|-------------------|
| `x + y` | Works | `ValueError` |
| `x * factor` | Works | Intersection |
| `x.add(y, join="inner")` | Works | Intersection |
| `x.add(y, join="outer")` | Works | Union with fill |
| `x <= rhs` (DataArray) | Works | `ValueError` |
| `x.le(rhs, join="inner")` | Works | Intersection |
| `x.le(rhs, join="left")` | Works | Left coords, NaN fill |